In [403]:
import spacy
# Need to run !python -m spacy download en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [404]:
from pathlib import Path as pathl
import sys, os, re
from heapq import nlargest
from collections import Counter

In [405]:
#append relevant file paths
new_path = pathl('.')
parent = new_path.resolve().parent
sys.path.append(str(parent))
from pdf_parser import pipeline
from collections import OrderedDict

In [410]:
data_path = parent / 'Data'
def get_text(filename):
    'Return text from a filename'
    pdf_file = data_path / filename
    text_dict = pipeline(filepath = str(pdf_file))
    text = list(text_dict.values())
    text = sum(text, [])
    text = [sentence.strip() for sentence in text]
    text = ' '.join(text)
    return text

In [411]:
nlp = spacy.load('en_core_web_sm')
punctuation = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~\n'

In [396]:
#Check to see if last file path in sys is the root folder
current_dir = os.walk(sys.path[-1] + '/Data')
all_text = [] #Term frequency
for file in next(current_dir)[-1]:
    try:
        text = get_text(file)
        all_text.append(process_text(text))
    except:
        pass

In [553]:
data_path = parent / 'Data'
def get_text(filename):
    'Return text from a filename'
    pdf_file = data_path / filename
    text_dict = pipeline(filepath = str(pdf_file))
    text = list(text_dict.values())
    text = sum(text, [])
    text = [sentence.strip() for sentence in text]
    text = ' '.join(text)
    return text

def lemmatizer(text):
    'Lemmatizes text'
    doc = nlp.pipe(text)
    lemmatized = []
    for sentence in doc:
        sent = []
        for word in sentence:
            if str(word) in punctuation:
                continue
            lemma = word.lemma_.strip() 
            sent.append(lemma)
        
        lemmatized.append(' '.join(sent))
    return lemmatized

def word_frequency(lemmatized_text):
    'Needs preprocessed text. Returns counts of words'
    bag_of_words = [sentence.split() for sentence in lemmatized_text]
    bag_of_words = sum(bag_of_words, [])
    return bag_of_words

def term_frequency_f(bag_of_words):
    'Calculates the term frequency (normalized) from a bag of words'
    word_frequency = Counter(bag_of_words)
    term_frequency_values = np.array(list(word_frequency.values())) / len(word_frequency)
    term_frequency = {}
    for word, value in zip(word_frequency.keys(), term_frequency_values):
        term_frequency[word] = value
    return term_frequency

def preprocess(filename):
    'Input a filename. Returns a term_frequency count'
    text = get_text(filename)
    text = text.split('.')
    lemmatized = lemmatizer(text)
    bag_of_words = word_frequency(lemmatized)
    term_frequency = term_frequency_f(bag_of_words)
    return term_frequency, bag_of_words

In [554]:
current_dir = os.walk(sys.path[-1] + '/Data')
files = []
for file in current_dir:
    files.append(file[-1])
files = sum(files, [])

texts = []
lexicon = [] #contains a list of words that exist in all files
for file in files:
    if file[-3:] == 'pdf':
        preprocess_output = preprocess(file)
        texts.append(preprocess_output[0])
        lexicon.append(preprocess_output[1])
lexicon = set(sum(lexicon, []))

In [562]:
document_frequency = []
for doc in texts:
    zero_vector = OrderedDict((token, 0) for token in lexicon)
    for key, value in zip(doc.keys(), doc.values()):
        zero_vector[key] = value
    document_frequency.append(zero_vector)
term_frequency = texts

In [599]:
document_tfidf_vectors = []
for doc in texts:
    zero_vector = OrderedDict((token, 0) for token in lexicon)
    for key, value in doc.items():
        docs_containing_key = 0
        for _doc in doc:
            if key in _doc:
                docs_containing_key += 1
        tf = value / len(lexicon)
        if docs_containing_key:
            idf = len(docs) / docs_containing_key
        else:
            idf = 0
        zero_vector[key] = tf * idf
    document_tfidf_vectors.append(zero_vector)

In [629]:
document_tfidf_vectors

[OrderedDict([('full', 0.00036199187302752667),
              ('passport', 0.00036199187302752667),
              ('an', 3.503147158330904e-05),
              ('sustainable', 0),
              ('prohibit', 0.00036199187302752667),
              ('Policies', 0),
              ('Director', 0.00036199187302752667),
              ('Group', 0),
              ('portuguese', 0),
              ('maintain', 0),
              ('see', 0.00036199187302752667),
              ('CoC', 0),
              ('financial', 0.00036199187302752667),
              ('confidential', 0),
              ('advance', 0),
              ('assess', 0),
              ('system', 0),
              ('ethical', 0),
              ('experience', 0),
              ('medtech', 0),
              ('Health', 0),
              ('safe', 0),
              ('Ltd', 0.0018099593651376333),
              ('SGPs', 0),
              ('encompass', 0),
              ('read', 0.00036199187302752667),
              ('6', 0.00018099593651376333)